In [1]:

import csv as csv
import numpy as np
import pandas
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold




In [2]:
#load in CSV files


#load in csv file train data
data = pandas.read_csv('train-2.csv', header = 0)

#load in csv file test data
data_test = pandas.read_csv('test-2.csv', header = 0)

In [3]:
data_test.head(10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = data['label']
labels=labels.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
label2 = pandas.DataFrame(enc.fit_transform(labels).toarray())
data2 = data.drop(['label'], axis=1)

/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [5]:
print data2.shape
print labels.shape
print data_test.shape


(42000, 784)
(42000, 1)
(28000, 784)


In [6]:
#create training and validation datasets

data2= np.array(data2,dtype=np.float32)
labels = np.array(label2,dtype = np.float32)
data_test = np.array(data_test, dtype = np.float32)
train_data, validation_data, train_labels, valid_labels = train_test_split(data2,labels, test_size = 0.15)






In [41]:
#Function
batch_size = 128
import tensorflow as tf

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, 784))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, 10))
    tf_valid_dataset = tf.constant(validation_data)
    tf_test_dataset = tf.constant(data_test)
    
    
    #Variables:
    
    weights = {'w1':tf.Variable(tf.truncated_normal([784,1024])), 'out':tf.Variable(tf.truncated_normal([1024,10]))}
    biases = { 'b1': tf.Variable(tf.zeros([1024])), 'out': tf.Variable(tf.zeros([10]))}
        
    def model(tf_train_dataset, weights, biases):
        hidden = tf.matmul(tf_train_dataset,weights['w1'])+biases['b1']
        hidden = tf.nn.relu(hidden)
        logit = tf.matmul(hidden, weights['out'])+biases['out']
        return logit
        
    #minimize cross entropy
    logit = model(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit, tf_train_labels))*.0001+0.00005*(tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(weights['w1']))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #variables for reporting predictions
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, weights, biases))


In [42]:
#definition of accuracy

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1)==np.argmax(labels,1))/predictions.shape[0])

results = np.zeros([28000,10])

with tf.Session(graph=graph) as session:
    #initialize variales
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for i in range(3001):
        offset = (batch_size*i)%(train_labels.shape[0] -batch_size)
        batch_data = train_data[offset:offset+batch_size,:]
        batch_labels = train_labels[offset:offset+batch_size,:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (i % 200 == 0):
            print('Loss at step %d: %f' % (i, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    results = test_prediction.eval()

        
     




Initialized
Loss at step 0: 23.938309
Training accuracy: 4.7%
Validation accuracy: 7.2%
Loss at step 200: 15.922574
Training accuracy: 81.2%
Validation accuracy: 80.4%
Loss at step 400: 15.665187
Training accuracy: 82.0%
Validation accuracy: 84.1%
Loss at step 600: 15.411307
Training accuracy: 86.7%
Validation accuracy: 86.4%
Loss at step 800: 15.211720
Training accuracy: 90.6%
Validation accuracy: 87.3%
Loss at step 1000: 15.026946
Training accuracy: 89.1%
Validation accuracy: 88.0%
Loss at step 1200: 14.989407
Training accuracy: 89.1%
Validation accuracy: 88.6%
Loss at step 1400: 14.844019
Training accuracy: 87.5%
Validation accuracy: 89.0%
Loss at step 1600: 14.594088
Training accuracy: 91.4%
Validation accuracy: 89.4%
Loss at step 1800: 14.573784
Training accuracy: 85.2%
Validation accuracy: 89.3%
Loss at step 2000: 14.327483
Training accuracy: 82.8%
Validation accuracy: 90.1%
Loss at step 2200: 14.116183
Training accuracy: 93.8%
Validation accuracy: 90.2%
Loss at step 2400: 13.962

In [43]:
results = pandas.DataFrame(results)
results['Index']=results.index +1
results['Final']=0



results.loc[(results[0]==1),'Final']=0
results.loc[(results[1]==1),'Final']=1
results.loc[(results[2]==1),'Final']=2
results.loc[(results[3]==1),'Final']=3
results.loc[(results[4]==1),'Final']=4
results.loc[(results[5]==1),'Final']=5
results.loc[(results[6]==1),'Final']=6
results.loc[(results[7]==1),'Final']=7
results.loc[(results[8]==1),'Final']=8
results.loc[(results[9]==1),'Final']=9


In [44]:

results.columns = ['00','01','02','03','04','05','06','07','08','09','Index','Final']
results = results.drop(['00','01','02','03','04','05','06','07','08','09'],axis=1)


print results.describe()



              Index         Final
count  28000.000000  28000.000000
mean   14000.500000      4.384071
std     8083.048105      2.856846
min        1.000000      0.000000
25%     7000.750000      2.000000
50%    14000.500000      4.000000
75%    21000.250000      7.000000
max    28000.000000      9.000000


In [45]:
np.savetxt("results.csv", results, delimiter = ',',fmt = '%.1i',comments='',header = "ImageID,Label")

